In [17]:
runner_path = "/home/alexmak123/neurone"
config_path = "/home/alexmak123/neurone/experiments/endonet_1class_plasmatic_search_168_v4/config.json"
engine_type = "gpu"
device_number = 4
tiles_directory_in_str = "/home/alexmak123/result_plasmatic_dataset_162_with_non_plasmatic_v2/test/images"
tiles_labels_directory_in_str = "/home/alexmak123/result_plasmatic_dataset_162_with_non_plasmatic_v2/test/labels"

%cd $runner_path

if (engine_type == "gpu"):
    #here we choose on which tesla to run
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = str(device_number)
    !export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'

import argparse
import json
from animus.torch.engine import CPUEngine, DDPEngine, DPEngine, GPUEngine, XLAEngine
from neurone.utils.general import load_yaml
import torch
from datetime import datetime 
import numpy as np
import matplotlib.pyplot as plt
import albumentations as A
from PIL import Image, ImageDraw
from skimage.io import imread, imsave, imshow
import cv2
%matplotlib inline

/home/alexmak123/neurone


Обучение

In [2]:
#Download our tiles
tiles_directory_in_str = "/home/alexmak123/plasmatic_dataset/dataset/images"
%cd $tiles_directory_in_str

index_image_dict = {}

alb_transforms = A.Compose([A.augmentations.Resize(1024, 1024)])
def import_tiles_from_directory_and_convert_them (tiles_directory_in_str):
    j = 0
    directory = os.fsencode(tiles_directory_in_str)
    converted_images = []
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if (filename.endswith(".png") or filename.endswith(".jpg")):
            basename = os.path.splitext(filename)[0]
            index_image_dict[j] = basename
            myTile = np.array(Image.open(filename).convert('RGB'))
            myTile = alb_transforms(image=myTile)["image"]
            converted_images.append(myTile)
            j += 1
            continue
        else:
            continue
    return np.array(converted_images)

converted_images = import_tiles_from_directory_and_convert_them (tiles_directory_in_str)
print(converted_images.shape)

/home/alexmak123/plasmatic_dataset/dataset/images
(41, 1024, 1024, 3)


In [3]:
#create a dict with coordinates of plasmatic cells of the image, validated by doctors
#Note: always false here, this code is for validation and accuracy count
we_have_Validation_Plasmatic_Cells = False

if (we_have_Validation_Plasmatic_Cells):
    validation_image_dict = {}
    tiles_directory_in_str = "/home/alexmak123/plasmatic_dataset/dataset/labels"
    %cd $tiles_directory_in_str
    directory = os.fsencode(tiles_directory_in_str)
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if (filename.endswith(".txt")):
            basename = os.path.splitext(filename)[0]
            validation_image_dict[basename] = []
            f = open(filename)
            for line in f:
                splited_line = line.split()
                if (splited_line[2] == '0'):
                    x = int(round(float(splited_line[0])))
                    y = int(round(float(splited_line[1])))
                    validation_image_dict[basename].append((x, y))
                    
    assert (len(index_image_dict) == len(validation_image_dict))
    result_dict = {k: validation_image_dict[v] for k, v in index_image_dict.items() if v in validation_image_dict}

In [4]:
# Runners
from neurone.runners.detection import BaseDetectionRunner, HeatmapDetectionRunner
from neurone.runners.segmentation import BaseSegmentationRunner
from neurone.runners.gridsearch import GridSearchRunner


E2E = {
    "cpu": CPUEngine,
    "gpu": GPUEngine,
    "dp": DPEngine,
    "ddp": DDPEngine,
    "xla": XLAEngine,
}


with open(config_path, "r") as config_file:
        config = json.load(config_file)
config["train"]["engine"] = engine_type

if config["train"]["deterministic"]:
    torch.use_deterministic_algorithms(config["train"]["deterministic"])
logging_dir = os.path.join(config["train"]["save_dir"], config["train"]["experiment_name"])
    
runner = globals()[config["model"]["runner"]](engine=E2E[engine_type](
    log_with="tensorboard", logging_dir=logging_dir), config=config)

runner._setup_model()
runner.model.eval()

HeatmapDetector(
  (heatmap_model): UnetPlusPlus(
    (encoder): ResNetEncoder(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

In [5]:
import torch
import torch.nn as nn

class ModifiedModel(nn.Module):
    def __init__(self, original_model, num_classes):
        super(ModifiedModel, self).__init__()
        self.original_model = original_model
        self.original_model.heatmap_model.segmentation_head[0] = nn.Conv2d(16, num_classes, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        
    def forward(self, x):
        original_output = self.original_model(x)
        return original_output

In [6]:
model = ModifiedModel(runner.model, 1)

In [7]:
print (model)

ModifiedModel(
  (original_model): HeatmapDetector(
    (heatmap_model): UnetPlusPlus(
      (encoder): ResNetEncoder(
        (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (layer1): Sequential(
          (0): Bottleneck(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn3): BatchNorm2d(256, eps=1e

In [ ]:
# Замораживаем все слои
"""
for param in model.parameters():
    param.requires_grad = False
    
# Размораживаем параметры последнего сверточного слоя
for param in model.original_model.heatmap_model.segmentation_head.parameters():
    param.requires_grad = True
"""

#runner.model = model
runner.run()

  0%|          | 0/130 [00:00<?, ?it/s]

  0%|          | 0/55 [00:00<?, ?it/s]

+----------------------------------+
| Epoch:   1/75       lr: 1.00E-04 |
| [                              ] |
| train                            |
| loss:  8.52E-04     metr: 0.0364 |
| valid                            |
| loss:  2.87E-04     metr: 0.2153 |
+----------------------------------+


  0%|          | 0/130 [00:00<?, ?it/s]

  0%|          | 0/55 [00:00<?, ?it/s]

...

+----------------------------------+
| Epoch:   7/75       lr: 1.00E-04 |
| [//                            ] |
| train                            |
| loss:  2.76E-05     metr: 0.7005 |
| valid                            |
| loss:  6.82E-05     metr: 0.4953 |
+----------------------------------+


  0%|          | 0/130 [00:00<?, ?it/s]

  0%|          | 0/55 [00:00<?, ?it/s]

+----------------------------------+
| Epoch:   8/75       lr: 1.00E-04 |
| [///                           ] |
| train                            |
| loss:  2.20E-05     metr: 0.6860 |
| valid                            |
| loss:  6.16E-05     metr: 0.5628 |
+----------------------------------+


  0%|          | 0/130 [00:00<?, ?it/s]

Применение обученной модели и подсчет точности

In [18]:
%cd $tiles_directory_in_str

index_image_dict = {}

alb_transforms = A.Compose([A.augmentations.Resize(1024, 1024)])
def import_tiles_from_directory_and_convert_them (tiles_directory_in_str):
    j = 0
    directory = os.fsencode(tiles_directory_in_str)
    converted_images = []
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if (filename.endswith(".png") or filename.endswith(".jpg")):
            basename = os.path.splitext(filename)[0]
            index_image_dict[j] = basename
            myTile = np.array(Image.open(filename).convert('RGB'))
            myTile = alb_transforms(image=myTile)["image"]
            converted_images.append(myTile)
            j += 1
            continue
        else:
            continue
    return np.array(converted_images)

converted_images = import_tiles_from_directory_and_convert_them (tiles_directory_in_str)

/home/alexmak123/result_plasmatic_dataset_162_with_non_plasmatic_v2/test/images


In [19]:
validation_image_dict = {}
for key_image, value_image in index_image_dict.items():
    validation_image_dict[value_image] = []

we_have_Validation_Plasmatic_Cells = True

if (we_have_Validation_Plasmatic_Cells):
    tiles_directory_in_str = tiles_labels_directory_in_str
    %cd $tiles_directory_in_str
    directory = os.fsencode(tiles_directory_in_str)
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if (filename.endswith(".txt")):
            basename = os.path.splitext(filename)[0]
            validation_image_dict[basename] = []
            f = open(filename)
            for line in f:
                splited_line = line.split()
                if (splited_line[2] == '0'):
                    x = int(round(float(splited_line[0])))
                    y = int(round(float(splited_line[1])))
                    validation_image_dict[basename].append((x, y))

/home/alexmak123/result_plasmatic_dataset_162_with_non_plasmatic_v2/test/labels


In [20]:
# Runners
from neurone.runners.detection import BaseDetectionRunner, HeatmapDetectionRunner
from neurone.runners.segmentation import BaseSegmentationRunner
from neurone.runners.gridsearch import GridSearchRunner


E2E = {
    "cpu": CPUEngine,
    "gpu": GPUEngine,
    "dp": DPEngine,
    "ddp": DDPEngine,
    "xla": XLAEngine,
}


with open(config_path, "r") as config_file:
        config = json.load(config_file)
config["train"]["engine"] = engine_type

if config["train"]["deterministic"]:
    torch.use_deterministic_algorithms(config["train"]["deterministic"])
logging_dir = os.path.join(config["train"]["save_dir"], config["train"]["experiment_name"])
    
runner = globals()[config["model"]["runner"]](engine=E2E[engine_type](
    log_with="tensorboard", logging_dir=logging_dir), config=config)

runner._setup_model()
runner.model.eval()

HeatmapDetector(
  (heatmap_model): UnetPlusPlus(
    (encoder): ResNetEncoder(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

In [21]:
#if we have gpu, we should run on cuda
if (engine_type == "gpu"):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    runner.model.to(device)
    
import sys
import time 

#our data to tensor
image_torch = torch.tensor(converted_images)
#image_torch = torch.unsqueeze(image_torch, 0)
image_torch = image_torch.permute(0, 3, 1, 2) 
image_torch = image_torch.float()
image_torch -= torch.tensor([config["data"]["norm_mean"][0], config["data"]["norm_mean"][1], config["data"]["norm_mean"][2]]).reshape(-1, 1, 1)
image_torch /= torch.tensor([config["data"]["norm_std"][0], config["data"]["norm_std"][1], config["data"]["norm_std"][2]]).reshape(-1, 1, 1)

if (engine_type == "cpu"):
    all_keypoints = runner.model(image_torch)["keypoints"]
    #all_confidences = runner.model(image_torch)["confidences"]

    
#amount of torch elements for a batch
#batch_size=2 is maximum for teslat4
batch_size = 2
if (engine_type == "gpu"):
    start = time.time() 
    all_keypoints = []
    #all_confidences = []
    splited_torch = torch.split(image_torch, batch_size)
    torch.cuda.empty_cache()
    i = 1
    num_iterations = int (image_torch.shape[0] / batch_size)
    
    for batch_torch in splited_torch:
        print ("iteration : ", i, "/", num_iterations)
        batch_torch = batch_torch.to(device)
        keypoint = np.array(runner.model(batch_torch)["keypoints"])
        all_keypoints.append(keypoint)
        #confidence = np.array(runner.model(batch_torch)["confidences"])
        #all_confidences.append(confidence) 
        i += 1
        del batch_torch
        
    end = time.time() - start
    print ("time, working on gpu", end)
    
#split keypoints of all tiles to array of selected colums with x_coords, y_coords and classes of all tiles
if (engine_type == "cpu"):
    j = 0
    selected_columns_array = []
    for index_tile in range(converted_images.shape[0]):
        selected_columns = []
        while (j < all_keypoints.shape[0] and all_keypoints[j][0] == index_tile):
            selected_columns.append([float(all_keypoints[j][1]), float(all_keypoints[j][2]), float(all_keypoints[j][3])])
            j += 1
        selected_columns_array.append(np.array(selected_columns))

if (engine_type == "gpu"):
    selected_columns_array = []
    for batch_keypoints in all_keypoints:
        j = 0
        for index_tile in range(batch_size):
            selected_columns = []
            while (j < batch_keypoints.shape[0] and batch_keypoints[j][0] == index_tile):
                selected_columns.append([float(batch_keypoints[j][1]), float(batch_keypoints[j][2]), float(batch_keypoints[j][3])])
                j += 1
            selected_columns_array.append(np.array(selected_columns))

iteration :  1 / 172
iteration :  2 / 172
iteration :  3 / 172
iteration :  4 / 172
iteration :  5 / 172
iteration :  6 / 172
iteration :  7 / 172
iteration :  8 / 172
iteration :  9 / 172
iteration :  10 / 172
iteration :  11 / 172
iteration :  12 / 172
iteration :  13 / 172
iteration :  14 / 172
iteration :  15 / 172
iteration :  16 / 172
iteration :  17 / 172
iteration :  18 / 172
iteration :  19 / 172
iteration :  20 / 172
iteration :  21 / 172
iteration :  22 / 172
iteration :  23 / 172
iteration :  24 / 172
iteration :  25 / 172
iteration :  26 / 172
iteration :  27 / 172
iteration :  28 / 172
iteration :  29 / 172
iteration :  30 / 172
iteration :  31 / 172
iteration :  32 / 172
iteration :  33 / 172
iteration :  34 / 172
iteration :  35 / 172
iteration :  36 / 172
iteration :  37 / 172
iteration :  38 / 172
iteration :  39 / 172
iteration :  40 / 172
iteration :  41 / 172
iteration :  42 / 172
iteration :  43 / 172
iteration :  44 / 172
iteration :  45 / 172
iteration :  46 / 1

In [29]:
import matplotlib.pyplot as plt
import numpy as np

print (len(converted_images))
algo_image_dict = {}
for img_index in range(len(converted_images)):
    algo_image_dict[img_index] = []
    
assert (len(index_image_dict) == len(validation_image_dict))
result_dict = {k: validation_image_dict[v] for k, v in index_image_dict.items() if v in validation_image_dict}
    
for i in range(len(converted_images)):
    # Create a new figure for each image
    #fig = plt.figure()
    
    # Draw circles on the image
    for dot_center in selected_columns_array[i]:
        if (dot_center[2] == 0):
            #cv2.circle(converted_images[i], (round(dot_center[0]), round(dot_center[1])), 4, (255,0,0), -1)
            algo_image_dict[i].append(((round(dot_center[0]), round(dot_center[1]))))


345


In [30]:
for key, value in algo_image_dict.items():
    print(key, value)

0 [(905, 946)]
1 [(213, 134)]
2 [(307, 252)]
3 []
4 [(531, 886), (975, 976), (800, 110)]
5 [(238, 402), (350, 461), (697, 464), (700, 552)]
6 [(925, 440)]
7 []
8 []
9 []
10 []
11 [(448, 326), (421, 478), (562, 896)]
12 []
13 [(375, 76), (1013, 376), (7, 226), (31, 1016)]
14 []
15 []
16 [(86, 716), (214, 509), (797, 442), (679, 177), (245, 411), (635, 628), (650, 368)]
17 [(641, 352), (516, 958), (459, 288)]
18 [(638, 545), (506, 690), (467, 420), (763, 466), (987, 344), (776, 636), (92, 203), (532, 604), (781, 432)]
19 [(193, 220), (8, 138), (214, 53)]
20 []
21 [(304, 296), (395, 589)]
22 [(971, 940), (71, 866), (201, 640)]
23 [(948, 370), (247, 330)]
24 []
25 [(688, 594), (268, 224)]
26 []
27 []
28 []
29 [(514, 714)]
30 [(428, 378), (899, 198)]
31 []
32 []
33 [(633, 460), (343, 520)]
34 []
35 [(461, 466), (547, 498), (803, 514), (323, 884)]
36 []
37 [(597, 374), (340, 972)]
38 [(712, 562), (899, 72), (521, 78), (586, 772), (182, 772), (259, 922), (426, 1018), (263, 820)]
39 []
40 []
4

In [31]:
import numpy as np
import copy


def is_close_enough(true_value_coord, pred_value_coord, threshold=15):
    x_diff = abs(true_value_coord[0] - pred_value_coord[0])
    y_diff = abs(true_value_coord[1] - pred_value_coord[1])
    return x_diff <= threshold and y_diff <= threshold


def calculate_tp_fp_fn_current_pic(true_values, predicted_values):
    num_true = len(true_values)
    num_predicted = len(predicted_values)
    tp = 0
    fp = 0
    fn = 0
    matched_true_values = set()

    for pred in predicted_values:
        found_match = False
        for true_value in true_values:
            if is_close_enough(true_value, pred) and true_value not in matched_true_values:
                tp += 1
                matched_true_values.add(true_value)
                found_match = True
                break
        if not found_match:
            fp += 1

    if (num_true != 0):
        fn = num_true - tp

    return tp, fp, fn


def get_precision_and_recall(algo_image_dict, result_dict):
    precision = 0.0
    recall = 0.0
    tp = 0.0
    fp = 0.0
    fn = 0.0
    for key in algo_image_dict:
        true_values = result_dict[key]
        predicted_values = algo_image_dict[key]
        if (len(true_values) > 0  or len(predicted_values) > 0):
            curr_tp, curr_fp, curr_fn = calculate_tp_fp_fn_current_pic(true_values, predicted_values)
            tp += curr_tp
            fp += curr_fp
            fn += curr_fn
            
    if (tp != 0 or fp != 0):
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1 = 2 * tp / (2 * tp + fp + fn)

    return precision, recall, f1


copy_algo_image_dict = copy.deepcopy(algo_image_dict)
copy_result_dict = copy.deepcopy(result_dict)
precision, recall, f1 = get_precision_and_recall(copy_algo_image_dict, copy_result_dict)
print ("precision, recall, f1 score")
print (precision, recall, f1)

precision, recall, f1 score
0.7440147329650092 0.8879120879120879 0.8096192384769539
